## Setup

In [1]:
%load_ext sql

In [2]:
from dotenv import find_dotenv
from dotenv import load_dotenv
load_dotenv(find_dotenv())

True

In [3]:
import google.auth
credentials, project_id = google.auth.default()

In [23]:
import os
from google.cloud.sql.connector import Connector
import sqlalchemy

# initialize Connector object
connector = Connector()

# function to return the database connection object
def get_conn():
    conn = connector.connect(
        INSTANCE_CONNECTION_NAME,
        "pg8000",
        user=DB_USER,
        password=DB_PASSWORD,
        db=DB_NAME
    )
    return conn

def get_pool():
    # create connection pool with 'creator' argument to our connection object function
    pool = sqlalchemy.create_engine(
        "postgresql+pg8000://",
        creator=get_conn,
    )
    
    return pool

## Create Business Connection

In [24]:
DB_USER = os.getenv("BUSINESS_USERNAME", "")
DB_PASSWORD = os.getenv("BUSINESS_PASSWORD", "")
DB_NAME = os.getenv("BUSINESS_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("BUSINESS_CONNECTION", "")

business_pool = get_pool()
%sql business_pool --alias business
%sql business

In [25]:
%%sql
select now();

Running query in 'business'

1 rows affected.

now
2024-04-29 03:20:34.387956+00:00


## Create Warehouse Connection

In [26]:
DB_USER = os.getenv("WAREHOUSE_USERNAME", "")
DB_PASSWORD = os.getenv("WAREHOUSE_PASSWORD", "")
DB_NAME = os.getenv("WAREHOUSE_NAME", "")
INSTANCE_CONNECTION_NAME = os.getenv("WAREHOUSE_CONNECTION", "")

warehouse_pool = get_pool()
%sql warehouse_pool --alias warehouse
%sql warehouse

In [27]:
%%sql
select now();

Running query in 'warehouse'

1 rows affected.

now
2024-04-29 03:22:30.802538+00:00


## List Database Connections

In [34]:
%sql --connections

current,url,alias
*,postgresql+pg8000://,business
,postgresql+pg8000://,warehouse


## Query Business Database

In [35]:
%sql business

In [36]:
%%sql business_data <<
select * from business

Running query in 'business'

2 rows affected.

In [37]:
business_data

id,legal_name,legal_type,identifier,tax_id,nano_id
1,C3 HUB CCC LTD. - IMPORT_TEST,CC,BC1120292,None,abcd123
2,FB HEALTH SUPPORT CCC CORP. - IMPORT_TEST,CC,BC1335427,None,efgh456


## Query Warehouse Database

In [38]:
%sql warehouse

Switching to connection 'warehouse'

In [40]:
%%sql colin_data <<
select * from colin.corporation limit 10;

Running query in 'warehouse'

10 rows affected.

In [41]:
colin_data

corp_num,corp_frozen_typ_cd,corp_typ_cd,recognition_dts,last_ar_filed_dt,transition_dt,bn_9,bn_15,accession_num,corp_password,prompt_question,admin_email,send_ar_ind,tilma_involved_ind,tilma_cessation_dt,firm_last_image_date,os_session,last_agm_date,firm_lp_xp_termination_date,last_ledger_dt,ar_reminder_option,ar_reminder_date,temp_password,temp_password_expiry_date
0003938,None,BC,None,None,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
A0003939,None,A,None,1983-01-21 00:00:00,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
0003939,None,BC,None,None,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
A0003940,None,A,None,None,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
0003940,None,BC,None,None,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
A0003941,None,A,None,None,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
0003941,None,BC,None,None,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
A0003942,None,A,None,1989-01-25 00:00:00,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
0003942,None,BC,None,None,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
A0003943,None,A,None,1980-12-31 00:00:00,None,None,None,None,None,None,None,Y,N,None,None,None,None,None,None,None,None,None,None
